In [1]:
import csv
import random
from collections import defaultdict
# Initialize dictionaries


# Read and shuffle the data
with open('100k_a.csv', 'r') as file:
    data = list(csv.reader(file))
    random.shuffle(data)

# Splitting the data: 80% train, 20% validation




In [2]:
streamer_interactions=defaultdict(int)
user_interactions=defaultdict(int)
for row in data:
    username=row[0]
    streamer=row[2]
    streamer_interactions[streamer]+=1
    user_interactions[username]+=1

small_streamer=set()
small_user=set()

for streamer in streamer_interactions.keys():
    if streamer_interactions[streamer]<3:
        small_streamer.add(streamer)
        
        
for username in user_interactions.keys():
    if user_interactions[username]<5:
        small_user.add(username)

In [3]:
new_data=[]

for row in data:
    username=row[0]
    streamer=row[2]
    if username in small_user or streamer in small_streamer:
        continue
    else:
        new_data.append(row)
    
    
    
split_index = int(0.8 * len(new_data))
train_data = new_data[:split_index]
validation_data = new_data[split_index:]

In [4]:
print(len(new_data))

2928857


In [5]:
# Function to process a single data row
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
streamer_viewers = defaultdict(int)
all_streamers=set()
# Process train data
for row in train_data:
    username = row[0]
    streamer = row[2]
    usersPerItem[streamer].add(username)
    itemsPerUser[username].add(streamer)
    streamer_viewers[streamer]+=1
    all_streamers.add(streamer)

all_streamers=list(all_streamers)



In [6]:
import random
dupe_count=0
balanced_valid=[]
for row in validation_data:
    username=row[0]
    streamer=row[2]
    while True:
        random_streamer = random.choice(all_streamers)
        balanced_valid.append([username,streamer,1])
        if random_streamer not in itemsPerUser[username]:
            balanced_valid.append([username, random_streamer,0])
            break


In [7]:
import statistics
mean_value = statistics.mean(streamer_viewers.values())
mean_value+=5
print("Mean Value:", mean_value)

Mean Value: 41.180494433378115


In [8]:
corr=0
false_positives=0
false_negatives=0
for row in balanced_valid:
    username=row[0]
    streamer=row[1]
    if streamer_viewers[streamer]>mean_value:
        pred=1
    else:
        pred=0
        
    if(row[2]==pred):
        
        corr+=1
    else:
        if(row[2]==1):
            false_positives+=1
        else:
            false_negatives+=1
            
print('baseline accuracy is',corr/len(balanced_valid))
print('false positives', false_positives, 'false_negatives',false_negatives)

baseline accuracy is 0.8521209938537007
false positives 115903 false_negatives 57376


In [9]:
print(len(balanced_valid))

1171762


In [10]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import tensorflow as tf





class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb

    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +                            tf.nn.l2_loss(self.gammaU) +                            tf.nn.l2_loss(self.gammaI))
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui

    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))


2023-12-02 18:42:26.687318: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
def get_popular_games(threshold):
    #game_count is a global dictionary
    mostPopular = [(streamer_viewers[x], x) for x in streamer_viewers]
    mostPopular.sort(reverse=True)

    temp=0
    popular_games = set()
    for times_played, game in mostPopular:
        temp += times_played
        popular_games.add(game)
        if temp > (len(train_data) * threshold):   # see if i am over the threshold
            return popular_games


userIDs={}
itemIDs={}
class BPR_Predictor:

    def __init__(self):
        self.interactions = []

    def fit(self, dataset, threshold=0.5, iters=200):
        self.popular_games = get_popular_games(threshold)
        #basically also from workbook all the way down to training step just renamed variables for me
        for i in train_data:
            user=i[0]
            game=i[2]
            if not user in userIDs: userIDs[user] = len(userIDs)
            if not game in itemIDs: itemIDs[game] = len(itemIDs)
            self.interactions.append((user,game,1))
                
        itemsPerUser = defaultdict(list)
        usersPerItem = defaultdict(list)
        items_keys=list(itemIDs.keys())
        for user, game, _ in self.interactions:
            itemsPerUser[user].append(game)
            usersPerItem[game].append(user)
        #below from workbook, changed names of variables for clarity
        def training_step(model, interactions):
            batch_size = 50000
            with tf.GradientTape() as tape:
                batch_user, batch_game, batch_item = [], [], []
                for _ in range(batch_size):
                    #pick a positive and negative sample
                    user,game,_ = random.choice(interactions) 
                    item_here = random.choice(items_keys) 
                    while item_here in itemsPerUser[user]:
                        item_here = random.choice(items_keys)
                    batch_user.append(userIDs[user])
                    batch_game.append(itemIDs[game])
                    batch_item.append(itemIDs[item_here])

                loss = model(batch_user,batch_game,batch_item)
                loss += model.reg()
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients((grad, var) for
                                    (grad, var) in zip(gradients, model.trainable_variables)
                                    if grad is not None)
            return loss.numpy()
        #below up to predict also from workbook
        optimizer = tf.keras.optimizers.Adam(0.1)
        self.modelBPR = BPRbatch(K=6, lamb= 0.00001)

        for i in range(iters):
            obj = training_step(self.modelBPR, self.interactions)
            if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))
    #below also from workbook        
    def predict(self, user, game, threshold=0.5):
        if user in userIDs and game in itemIDs: # add this because user or item id not in lists we made
            pred = self.modelBPR.predict(userIDs[user], itemIDs[game]).numpy()
            if(pred>threshold):
                return 1
            return 0
        else:
            if(game in self.popular_games):
                return 1
            return 0


In [23]:
#model = BPR_Predictor()


#model.fit(train_data) #train on whole set lol

correct = 0
tot=0
false_positives=0
false_negatives=0
for user, game, review in balanced_valid:
    tot+=1
    pred = model.predict(user, game)
    if pred == review:
        correct += 1
    elif(pred==1 and review==0):
        false_positives+=1
    else:
        false_negatives+=1
    #print(f"BPR_Predictor accuracy: ", correct / tot,false_positives,false_negatives)
print(f"BPR_Predictor accuracy: ", correct / len(balanced_valid),false_positives,false_negatives)

BPR_Predictor accuracy:  0.8472676191922933 47619 131347
